In [1]:
from pyspark.sql import Row
l = [(1001,'Apple Mac Book', 11, 'I'),(1002,'Apple iPhone 11', 22, 'I'),(1003,'Redmi Note 4', 33, 'I'),(1004, 'Dell Inspiron', 44, 'I')]
rdd = sc.parallelize(l)
schemaRdd = rdd.map(lambda x: Row(product_id=int(x[0]), product_name=x[1], stocks_left=int(x[2]), row_flag=x[3]))
product_df = spark.createDataFrame(schemaRdd)
display(product_df)

product_id,product_name,row_flag,stocks_left
1001,Apple Mac Book,I,11
1002,Apple iPhone 11,I,22
1003,Redmi Note 4,I,33
1004,Dell Inspiron,I,44


In [2]:
database_name = 'delta_demo'
delta_table_name = 'product_table_delta'
parquet_table_name = 'product_table_parquet'

In [3]:
(product_df
  .repartition(2)
  .write
  .mode("overwrite")
  .format("parquet")
  .saveAsTable(f'{database_name}.{parquet_table_name}')
)

(product_df
  .repartition(2)
  .write
  .mode("overwrite")
  .option("overwriteSchema", True)
  .format("delta")
  .saveAsTable(f'{database_name}.{delta_table_name}')
)

In [4]:
%sql

desc formatted delta_demo.product_table_delta

col_name,data_type,comment
product_id,bigint,null
product_name,string,null
row_flag,string,null
stocks_left,bigint,null
,,
# Detailed Table Information,,
Database,delta_demo,
Table,product_table_delta,
Owner,root,
Created Time,Sun Jun 28 07:40:09 UTC 2020,


In [5]:
%fs

ls dbfs:/user/hive/warehouse/delta_demo.db/product_table_delta/_delta_log

path,name,size
dbfs:/user/hive/warehouse/delta_demo.db/product_table_delta/_delta_log/.s3-optimization-0,.s3-optimization-0,0
dbfs:/user/hive/warehouse/delta_demo.db/product_table_delta/_delta_log/.s3-optimization-1,.s3-optimization-1,0
dbfs:/user/hive/warehouse/delta_demo.db/product_table_delta/_delta_log/.s3-optimization-2,.s3-optimization-2,0
dbfs:/user/hive/warehouse/delta_demo.db/product_table_delta/_delta_log/00000000000000000000.crc,00000000000000000000.crc,89
dbfs:/user/hive/warehouse/delta_demo.db/product_table_delta/_delta_log/00000000000000000000.json,00000000000000000000.json,3287
dbfs:/user/hive/warehouse/delta_demo.db/product_table_delta/_delta_log/00000000000000000001.crc,00000000000000000001.crc,89
dbfs:/user/hive/warehouse/delta_demo.db/product_table_delta/_delta_log/00000000000000000001.json,00000000000000000001.json,3414
dbfs:/user/hive/warehouse/delta_demo.db/product_table_delta/_delta_log/00000000000000000002.crc,00000000000000000002.crc,91
dbfs:/user/hive/warehouse/delta_demo.db/product_table_delta/_delta_log/00000000000000000002.json,00000000000000000002.json,2700
dbfs:/user/hive/warehouse/delta_demo.db/product_table_delta/_delta_log/00000000000000000003.crc,00000000000000000003.crc,91


In [6]:
sqlCmd = f"SELECT * FROM {database_name}.{parquet_table_name}"
display(spark.sql(sqlCmd))

product_id,product_name,row_flag,stocks_left
1002,Apple iPhone 11,I,22
1001,Apple Mac Book,I,11
1004,Dell Inspiron,I,44
1003,Redmi Note 4,I,33


In [7]:
sqlCmd = f"SELECT * FROM {database_name}.{delta_table_name}"
display(spark.sql(sqlCmd))

product_id,product_name,row_flag,stocks_left
1002,Apple iPhone 11,I,22
1001,Apple Mac Book,I,11
1004,Dell Inspiron,I,44
1003,Redmi Note 4,I,33


In [8]:
l = [(1001,'Apple Mac Book', 'eleven', 'I'),(1002,'Apple iPhone 11', 22.52, 'I'),(1003,'Redmi Note 4', 33, 'I'),(1004, 'Dell Inspiron', 44, 'I')]
rdd = sc.parallelize(l)
schemaRdd = rdd.map(lambda x: Row(product_id=int(x[0]), product_name=x[1], stocks_left=str(x[2]), row_flag=x[3]))
type_changed_product_df = spark.createDataFrame(schemaRdd)
display(type_changed_product_df)

product_id,product_name,row_flag,stocks_left
1001,Apple Mac Book,I,eleven
1002,Apple iPhone 11,I,22.52
1003,Redmi Note 4,I,33
1004,Dell Inspiron,I,44


In [9]:
(type_changed_product_df
  .repartition(2)
  .write
  .mode("append")
  .format("parquet")
  .saveAsTable(f'{database_name}.{parquet_table_name}')
)

In [10]:
(type_changed_product_df
  .repartition(2)
  .write
  .mode("append")
  .format("delta")
  .saveAsTable(f'{database_name}.{delta_table_name}')
)

--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 62 try : 
 ---> 63 return f ( * a , ** kw ) 
 64 except py4j . protocol . Py4JJavaError as e : 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 --> 328 format(target_id, ".", name), value)
 329 else : 

 Py4JJavaError : An error occurred while calling o1955.saveAsTable.
: org.apache.spark.sql.AnalysisException: Failed to merge fields 'stocks_left' and 'stocks_left'. Failed to merge incompatible data types LongType and DoubleType;;
	at com.databricks.sql.transaction.tahoe.schema.SchemaUtils$$anonfun$18.apply(SchemaUtils.scala:685)
	at com.databricks.sql.transaction.tahoe.schema.SchemaUtils$$anonfun$18.apply(SchemaUtils.scala:674)
	at scala.collection.TraversableLike$$anonfun$map$1.apply(TraversableLike.scala:234)
	at scala.collection.TraversableLike$$anonfun$map$1.apply(TraversableLike.scala:234)
	at scala.collection.IndexedSeqOptimized$class.foreach(IndexedSeqOptimized.scala:33)
	at scala.collection.mutable.ArrayOps$ofRef.foreach(ArrayOps.scala:186)
	at scala.collection.TraversableLike$class.map(TraversableLike.scala:234)
	at scala.collection.mutable.ArrayOps$ofRef.map(ArrayOps.scala:186)
	at com.databricks.sql.transaction.tahoe.schema.SchemaUtils$.com$databricks$sql$transaction$tahoe$schema$SchemaUtils$$merge$1(SchemaUtils.scala:674)
	at com.databricks.sql.transaction.tahoe.schema.SchemaUtils$.mergeSchemas(SchemaUtils.scala:750)
	at com.databricks.sql.transaction.tahoe.schema.ImplicitMetadataOperation$class.updateMetadata(ImplicitMetadataOperation.scala:64)
	at com.databricks.sql.transaction.tahoe.commands.WriteIntoDelta.updateMetadata(WriteIntoDelta.scala:50)
	at com.databricks.sql.transaction.tahoe.commands.WriteIntoDelta.write(WriteIntoDelta.scala:91)
	at com.databricks.sql.transaction.tahoe.commands.CreateDeltaTableCommand$$anonfun$run$2.apply(CreateDeltaTableCommand.scala:119)
	at com.databricks.sql.transaction.tahoe.commands.CreateDeltaTableCommand$$anonfun$run$2.apply(CreateDeltaTableCommand.scala:93)
	at com.databricks.logging.UsageLogging$$anonfun$recordOperation$1.apply(UsageLogging.scala:428)
	at com.databricks.logging.UsageLogging$$anonfun$withAttributionContext$1.apply(UsageLogging.scala:238)
	at scala.util.DynamicVariable.withValue(DynamicVariable.scala:58)
	at com.databricks.logging.UsageLogging$class.withAttributionContext(UsageLogging.scala:233)
	at com.databricks.spark.util.PublicDBLogging.withAttributionContext(DatabricksSparkUsageLogger.scala:18)
	at com.databricks.logging.UsageLogging$class.withAttributionTags(UsageLogging.scala:275)
	at com.databricks.spark.util.PublicDBLogging.withAttributionTags(DatabricksSparkUsageLogger.scala:18)
	at com.databricks.logging.UsageLogging$class.recordOperation(UsageLogging.scala:409)
	at com.databricks.spark.util.PublicDBLogging.recordOperation(DatabricksSparkUsageLogger.scala:18)
	at com.databricks.spark.util.PublicDBLogging.recordOperation0(DatabricksSparkUsageLogger.scala:55)
	at com.databricks.spark.util.DatabricksSparkUsageLogger.recordOperation(DatabricksSparkUsageLogger.scala:98)
	at com.databricks.spark.util.UsageLogger$class.recordOperation(UsageLogger.scala:67)
	at com.databricks.spark.util.DatabricksSparkUsageLogger.recordOperation(DatabricksSparkUsageLogger.scala:67)
	at com.databricks.spark.util.UsageLogging$class.recordOperation(UsageLogger.scala:342)
	at com.databricks.sql.transaction.tahoe.commands.CreateDeltaTableCommand.recordOperation(CreateDeltaTableCommand.scala:45)
	at com.databricks.sql.transaction.tahoe.metering.DeltaLogging$class.recordDeltaOperation(DeltaLogging.scala:108)
	at com.databricks.sql.transaction.tahoe.commands.CreateDeltaTableCommand.recordDeltaOperation(CreateDeltaTableCommand.scala:45)
	at com.databricks

In [11]:
sqlCmd = f"SELECT * FROM {database_name}.{delta_table_name}"
display(spark.sql(sqlCmd))

product_id,product_name,row_flag,stocks_left
1002,Apple iPhone 11,I,22
1002,Apple iPhone 11,I,22
1001,Apple Mac Book,I,11
1001,Apple Mac Book,I,11
1004,Dell Inspiron,I,44
1004,Dell Inspiron,I,44
1003,Redmi Note 4,I,33
1003,Redmi Note 4,I,33


In [12]:
sqlCmd = f"SELECT * FROM {database_name}.{parquet_table_name}"
display(spark.sql(sqlCmd))

product_id,product_name,row_flag,stocks_left
1002,Apple iPhone 11,I,22
1002,Apple iPhone 11,I,22
1002,Apple iPhone 11,I,22
1002,Apple iPhone 11,I,22
1002,Apple iPhone 11,I,22
1001,Apple Mac Book,I,11
1001,Apple Mac Book,I,11
1001,Apple Mac Book,I,11
1001,Apple Mac Book,I,11
1004,Dell Inspiron,I,44


In [15]:
l = [(1001,'Apple Mac Book', 11, 'I', 'new_1'),(1002,'Apple iPhone 11', 22, 'I', 'new_2'),(1003,'Redmi Note 4', 33, 'I', 'new_3'),(1004, 'Dell Inspiron', 44, 'I', 'new_4')]
rdd = sc.parallelize(l)
schemaRdd = rdd.map(lambda x: Row(product_id=int(x[0]), product_name=x[1], stocks_left=int(x[2]), row_flag=x[3], new_col=x[4]))
new_col_product_df = spark.createDataFrame(schemaRdd)
display(new_col_product_df)

new_col,product_id,product_name,row_flag,stocks_left
new_1,1001,Apple Mac Book,I,11
new_2,1002,Apple iPhone 11,I,22
new_3,1003,Redmi Note 4,I,33
new_4,1004,Dell Inspiron,I,44


In [16]:
new_col_product_df.printSchema()

root
-- new_col: string (nullable = true)
-- product_id: long (nullable = true)
-- product_name: string (nullable = true)
-- row_flag: string (nullable = true)
-- stocks_left: long (nullable = true)

In [17]:
(new_col_product_df
  .repartition(2)
  .write
  .mode("append")
  .format("parquet")
  .saveAsTable(f'{database_name}.{parquet_table_name}')
)

--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 62 try : 
 ---> 63 return f ( * a , ** kw ) 
 64 except py4j . protocol . Py4JJavaError as e : 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 --> 328 format(target_id, ".", name), value)
 329 else : 

 Py4JJavaError : An error occurred while calling o2742.saveAsTable.
: org.apache.spark.sql.AnalysisException: The column number of the existing table delta_demo.product_table_parquet(struct<product_id:bigint,product_name:string,row_flag:string,stocks_left:bigint>) doesn't match the data schema(struct<new_col:string,product_id:bigint,product_name:string,row_flag:string,stocks_left:string>);
	at org.apache.spark.sql.execution.datasources.PreprocessTableCreation$$anonfun$apply$2.applyOrElse(rules.scala:197)
	at org.apache.spark.sql.execution.datasources.PreprocessTableCreation$$anonfun$apply$2.applyOrElse(rules.scala:138)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper$$anonfun$resolveOperatorsDown$1$$anonfun$2.apply(AnalysisHelper.scala:108)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper$$anonfun$resolveOperatorsDown$1$$anonfun$2.apply(AnalysisHelper.scala:108)
	at org.apache.spark.sql.catalyst.trees.CurrentOrigin$.withOrigin(TreeNode.scala:76)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper$$anonfun$resolveOperatorsDown$1.apply(AnalysisHelper.scala:107)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper$$anonfun$resolveOperatorsDown$1.apply(AnalysisHelper.scala:106)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper$.allowInvokingTransformsInAnalyzer(AnalysisHelper.scala:194)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper$class.resolveOperatorsDown(AnalysisHelper.scala:106)
	at org.apache.spark.sql.catalyst.plans.logical.LogicalPlan.resolveOperatorsDown(LogicalPlan.scala:29)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper$class.resolveOperators(AnalysisHelper.scala:73)
	at org.apache.spark.sql.catalyst.plans.logical.LogicalPlan.resolveOperators(LogicalPlan.scala:29)
	at org.apache.spark.sql.execution.datasources.PreprocessTableCreation.apply(rules.scala:138)
	at org.apache.spark.sql.execution.datasources.PreprocessTableCreation.apply(rules.scala:134)
	at org.apache.spark.sql.catalyst.rules.RuleExecutor$$anonfun$execute$1$$anonfun$apply$1.apply(RuleExecutor.scala:112)
	at org.apache.spark.sql.catalyst.rules.RuleExecutor$$anonfun$execute$1$$anonfun$apply$1.apply(RuleExecutor.scala:109)
	at scala.collection.LinearSeqOptimized$class.foldLeft(LinearSeqOptimized.scala:124)
	at scala.collection.immutable.List.foldLeft(List.scala:84)
	at org.apache.spark.sql.catalyst.rules.RuleExecutor$$anonfun$execute$1.apply(RuleExecutor.scala:109)
	at org.apache.spark.sql.catalyst.rules.RuleExecutor$$anonfun$execute$1.apply(RuleExecutor.scala:101)
	at scala.collection.immutable.List.foreach(List.scala:392)
	at org.apache.spark.sql.catalyst.rules.RuleExecutor.execute(RuleExecutor.scala:101)
	at org.apache.spark.sql.catalyst.analysis.Analyzer.org$apache$spark$sql$catalyst$analysis$Analyzer$$executeSameContext(Analyzer.scala:137)
	at org.apache.spark.sql.catalyst.analysis.Analyzer.execute(Analyzer.scala:131)
	at org.apache.spark.sql.catalyst.analysis.Analyzer.execute(Analyzer.scala:103)
	at org.apache.spark.sql.catalyst.rules.RuleExecutor$$anonfun$executeAndTrack$1.apply(RuleExecutor.scala:80)
	at org.apache.spark.sql.catalyst.rules.RuleExecutor$$anonfun$executeAndTrack$1.apply(RuleExecutor.scala:80)
	at org.apache.spark.sql.catalyst.QueryPlanningTracker$.withTracker(QueryPlanningTracker.scala:88)
	at org.apache.spark.sql.catalyst.rules.RuleExecutor.executeAndTrack(RuleExecutor.scala:79)
	at org.apache.spark.sql.catalyst.analys

In [18]:
(new_col_product_df
  .repartition(2)
  .write
  .mode("append")
  .format("delta")
  .saveAsTable(f'{database_name}.{delta_table_name}')
)

--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 62 try : 
 ---> 63 return f ( * a , ** kw ) 
 64 except py4j . protocol . Py4JJavaError as e : 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 --> 328 format(target_id, ".", name), value)
 329 else : 

 Py4JJavaError : An error occurred while calling o3662.saveAsTable.
: org.apache.spark.sql.AnalysisException: A schema mismatch detected when writing to the Delta table (Table ID: 59535559-84e1-46da-a566-06c753305e5a).
To enable schema migration, please set:
'.option("mergeSchema", "true")'.

Table schema:
root
-- product_id: long (nullable = true)
-- product_name: string (nullable = true)
-- row_flag: string (nullable = true)
-- stocks_left: long (nullable = true)


Data schema:
root
-- new_col: string (nullable = true)
-- product_id: long (nullable = true)
-- product_name: string (nullable = true)
-- row_flag: string (nullable = true)
-- stocks_left: long (nullable = true)

 
If Table ACLs are enabled, these options will be ignored. Please use the ALTER TABLE
command for changing the schema.
 ;
	at com.databricks.sql.transaction.tahoe.MetadataMismatchErrorBuilder.finalizeAndThrow(DeltaErrors.scala:948)
	at com.databricks.sql.transaction.tahoe.schema.ImplicitMetadataOperation$class.updateMetadata(ImplicitMetadataOperation.scala:125)
	at com.databricks.sql.transaction.tahoe.commands.WriteIntoDelta.updateMetadata(WriteIntoDelta.scala:50)
	at com.databricks.sql.transaction.tahoe.commands.WriteIntoDelta.write(WriteIntoDelta.scala:91)
	at com.databricks.sql.transaction.tahoe.commands.CreateDeltaTableCommand$$anonfun$run$2.apply(CreateDeltaTableCommand.scala:119)
	at com.databricks.sql.transaction.tahoe.commands.CreateDeltaTableCommand$$anonfun$run$2.apply(CreateDeltaTableCommand.scala:93)
	at com.databricks.logging.UsageLogging$$anonfun$recordOperation$1.apply(UsageLogging.scala:428)
	at com.databricks.logging.UsageLogging$$anonfun$withAttributionContext$1.apply(UsageLogging.scala:238)
	at scala.util.DynamicVariable.withValue(DynamicVariable.scala:58)
	at com.databricks.logging.UsageLogging$class.withAttributionContext(UsageLogging.scala:233)
	at com.databricks.spark.util.PublicDBLogging.withAttributionContext(DatabricksSparkUsageLogger.scala:18)
	at com.databricks.logging.UsageLogging$class.withAttributionTags(UsageLogging.scala:275)
	at com.databricks.spark.util.PublicDBLogging.withAttributionTags(DatabricksSparkUsageLogger.scala:18)
	at com.databricks.logging.UsageLogging$class.recordOperation(UsageLogging.scala:409)
	at com.databricks.spark.util.PublicDBLogging.recordOperation(DatabricksSparkUsageLogger.scala:18)
	at com.databricks.spark.util.PublicDBLogging.recordOperation0(DatabricksSparkUsageLogger.scala:55)
	at com.databricks.spark.util.DatabricksSparkUsageLogger.recordOperation(DatabricksSparkUsageLogger.scala:98)
	at com.databricks.spark.util.UsageLogger$class.recordOperation(UsageLogger.scala:67)
	at com.databricks.spark.util.DatabricksSparkUsageLogger.recordOperation(DatabricksSparkUsageLogger.scala:67)
	at com.databricks.spark.util.UsageLogging$class.recordOperation(UsageLogger.scala:342)
	at com.databricks.sql.transaction.tahoe.commands.CreateDeltaTableCommand.recordOperation(CreateDeltaTableCommand.scala:45)
	at com.databricks.sql.transaction.tahoe.metering.DeltaLogging$class.recordDeltaOperation(DeltaLogging.scala:108)
	at com.databricks.sql.transaction.tahoe.commands.CreateDeltaTableCommand.recordDeltaOperation(CreateDeltaTableCommand.scala:45)
	at com.databricks.sql.transaction.tahoe.commands.CreateDeltaTableCommand.run(CreateDeltaTableCommand.scala:93)
	at org.apache.spark.sql.execution.command.ExecutedCommandExec.sideEffectResult$lzycompute(commands.scala:70)
	at org.apache.spark.sql.e

In [19]:
(new_col_product_df
  .repartition(2)
  .write
  .mode("append")
  .option("mergeSchema", True)
  .format("delta")
  .saveAsTable(f'{database_name}.{delta_table_name}')
)

In [20]:
sqlCmd = f"SELECT * FROM {database_name}.{parquet_table_name}"
display(spark.sql(sqlCmd))

product_id,product_name,row_flag,stocks_left
1002,Apple iPhone 11,I,22
1002,Apple iPhone 11,I,22
1002,Apple iPhone 11,I,22
1002,Apple iPhone 11,I,22
1002,Apple iPhone 11,I,22
1001,Apple Mac Book,I,11
1001,Apple Mac Book,I,11
1001,Apple Mac Book,I,11
1001,Apple Mac Book,I,11
1004,Dell Inspiron,I,44


In [21]:
sqlCmd = f"SELECT * FROM {database_name}.{delta_table_name}"
display(spark.sql(sqlCmd))

product_id,product_name,row_flag,stocks_left,new_col
1002,Apple iPhone 11,I,22,new_2
1001,Apple Mac Book,I,11,new_1
1004,Dell Inspiron,I,44,new_4
1003,Redmi Note 4,I,33,new_3
1002,Apple iPhone 11,I,22,null
1001,Apple Mac Book,I,11,null
1004,Dell Inspiron,I,44,null
1003,Redmi Note 4,I,33,null


### Changing a column’s type or name or dropping a column requires rewriting the table. To do this, use the overwriteSchema option:

# CDC USE CASE - DML/DDL

In [29]:
l = [(1001,'Apple Mac Book', 110, 'U', 'new_1'),(1002,'Apple iPhone 11', 2, 'U', 'new_2'),(1003,'Redmi Note 4', 3, 'U', 'new_3'),(1004, 'Dell Inspiron', 0, 'D', 'new_4')]
rdd = sc.parallelize(l)
schemaRdd = rdd.map(lambda x: Row(product_id=int(x[0]), product_name=x[1], stocks_left=int(x[2]), row_flag=x[3], new_col=x[4]))
merge_product_df = spark.createDataFrame(schemaRdd)
display(merge_product_df)

new_col,product_id,product_name,row_flag,stocks_left
new_1,1001,Apple Mac Book,U,110
new_2,1002,Apple iPhone 11,U,2
new_3,1003,Redmi Note 4,U,3
new_4,1004,Dell Inspiron,D,0


In [30]:
merge_product_df.createOrReplaceTempView('source_table')

In [32]:
spark.sql("""
          MERGE INTO delta_demo.product_table_delta target_table
          USING source_table
          ON source_table.product_id = target_table.product_id
          WHEN MATCHED AND source_table.row_flag = 'U'
          THEN
          UPDATE SET *
          WHEN MATCHED AND source_table.row_flag = 'D'
          THEN DELETE
          WHEN NOT MATCHED and source_table.row_flag = 'I'
          THEN INSERT *
          """)

Out[24]: DataFrame[]

In [33]:
sqlCmd = f"SELECT * FROM {database_name}.{delta_table_name}"
display(spark.sql(sqlCmd))

product_id,product_name,row_flag,stocks_left,new_col
1002,Apple iPhone 11,U,2,new_2
1002,Apple iPhone 11,U,2,new_2
1001,Apple Mac Book,U,110,new_1
1001,Apple Mac Book,U,110,new_1
1003,Redmi Note 4,U,3,new_3
1003,Redmi Note 4,U,3,new_3


# SHOW I/U/D counts, this features was not there previously. If possible also show the slack channel screenshot

In [36]:
%sql

describe history delta_demo.product_table_delta

-- Table history is retained for 30 days

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics
7,2020-06-28T10:52:13.000+0000,1583409609571278,techieeisland@gmail.com,MERGE,"Map(predicate -> (source_table.`product_id` = target_table.`product_id`), updatePredicate -> (source_table.`row_flag` = 'U'), deletePredicate -> (source_table.`row_flag` = 'D'), insertPredicate -> (source_table.`row_flag` = 'I'))",null,List(891795827403111),0628-102648-pick544,6,WriteSerializable,false,"Map(numTargetRowsCopied -> 0, numTargetRowsDeleted -> 2, numFiles -> 4, numTargetFilesAfterSkipping -> 10, numTargetFilesAdded -> 4, numTargetRowsInserted -> 0, numTargetRowsUpdated -> 6, numOutputRows -> 6, numParts -> 0, numOutputBytes -> 5583, numSourceRows -> 4, numTargetFilesRemoved -> 8, numTargetFilesBeforeSkipping -> 10)"
6,2020-06-28T10:41:40.000+0000,1583409609571278,techieeisland@gmail.com,WRITE,"Map(mode -> Append, partitionBy -> [])",null,List(891795827403111),0628-102648-pick544,5,WriteSerializable,true,"Map(numFiles -> 5, numOutputBytes -> 6618, numOutputRows -> 4, numParts -> 0)"
5,2020-06-28T10:38:26.000+0000,1583409609571278,techieeisland@gmail.com,WRITE,"Map(mode -> Overwrite, partitionBy -> [])",null,List(891795827403111),0628-102648-pick544,4,WriteSerializable,false,"Map(numFiles -> 5, numOutputBytes -> 5602, numOutputRows -> 4, numParts -> 0)"
4,2020-06-28T10:36:48.000+0000,1583409609571278,techieeisland@gmail.com,WRITE,"Map(mode -> Overwrite, partitionBy -> [])",null,List(891795827403111),0628-102648-pick544,3,WriteSerializable,false,"Map(numFiles -> 5, numOutputBytes -> 5602, numOutputRows -> 4, numParts -> 0)"
3,2020-06-28T08:17:46.000+0000,1583409609571278,techieeisland@gmail.com,WRITE,"Map(mode -> Append, partitionBy -> [])",null,List(891795827403111),0628-070251-hove428,2,WriteSerializable,true,"Map(numFiles -> 5, numOutputBytes -> 6618, numOutputRows -> 4, numParts -> 0)"
2,2020-06-28T08:00:00.000+0000,1583409609571278,techieeisland@gmail.com,WRITE,"Map(mode -> Append, partitionBy -> [])",null,List(891795827403111),0628-070251-hove428,1,WriteSerializable,true,"Map(numFiles -> 5, numOutputBytes -> 5602, numOutputRows -> 4, numParts -> 0)"
1,2020-06-28T07:44:11.000+0000,1583409609571278,techieeisland@gmail.com,WRITE,"Map(mode -> Overwrite, partitionBy -> [])",null,List(891795827403111),0628-070251-hove428,0,WriteSerializable,false,"Map(numFiles -> 5, numOutputBytes -> 5602, numOutputRows -> 4, numParts -> 0)"
0,2020-06-28T07:40:12.000+0000,1583409609571278,techieeisland@gmail.com,WRITE,"Map(mode -> Overwrite, partitionBy -> [])",null,List(891795827403111),0628-070251-hove428,null,WriteSerializable,false,"Map(numFiles -> 5, numOutputBytes -> 5602, numOutputRows -> 4, numParts -> 0)"


In [37]:
%sql

select operationMetrics.numTargetRowsInserted, operationMetrics.numTargetRowsUpdated, operationMetrics.numTargetRowsDeleted from (describe history delta_demo.product_table_delta limit 1)

numTargetRowsInserted,numTargetRowsUpdated,numTargetRowsDeleted
0,6,2


# TIME TRAVEL

In [41]:
%sql

select * from delta_demo.product_table_delta version as of 1

product_id,product_name,row_flag,stocks_left
1002,Apple iPhone 11,I,22
1001,Apple Mac Book,I,11
1004,Dell Inspiron,I,44
1003,Redmi Note 4,I,33


# SHOW TRANSACTIONAL LOG FILES

In [45]:
%fs

ls dbfs:/user/hive/warehouse/delta_demo.db/product_table_delta/_delta_log

path,name,size
dbfs:/user/hive/warehouse/delta_demo.db/product_table_delta/_delta_log/.s3-optimization-0,.s3-optimization-0,0
dbfs:/user/hive/warehouse/delta_demo.db/product_table_delta/_delta_log/.s3-optimization-1,.s3-optimization-1,0
dbfs:/user/hive/warehouse/delta_demo.db/product_table_delta/_delta_log/.s3-optimization-2,.s3-optimization-2,0
dbfs:/user/hive/warehouse/delta_demo.db/product_table_delta/_delta_log/00000000000000000000.crc,00000000000000000000.crc,89
dbfs:/user/hive/warehouse/delta_demo.db/product_table_delta/_delta_log/00000000000000000000.json,00000000000000000000.json,3287
dbfs:/user/hive/warehouse/delta_demo.db/product_table_delta/_delta_log/00000000000000000001.crc,00000000000000000001.crc,89
dbfs:/user/hive/warehouse/delta_demo.db/product_table_delta/_delta_log/00000000000000000001.json,00000000000000000001.json,3414
dbfs:/user/hive/warehouse/delta_demo.db/product_table_delta/_delta_log/00000000000000000002.crc,00000000000000000002.crc,91
dbfs:/user/hive/warehouse/delta_demo.db/product_table_delta/_delta_log/00000000000000000002.json,00000000000000000002.json,2700
dbfs:/user/hive/warehouse/delta_demo.db/product_table_delta/_delta_log/00000000000000000003.crc,00000000000000000003.crc,91


In [46]:
%fs

ls dbfs:/user/hive/warehouse/delta_demo.db/product_table_delta/_delta_log/

path,name,size
dbfs:/user/hive/warehouse/delta_demo.db/product_table_delta/_delta_log/.s3-optimization-0,.s3-optimization-0,0
dbfs:/user/hive/warehouse/delta_demo.db/product_table_delta/_delta_log/.s3-optimization-1,.s3-optimization-1,0
dbfs:/user/hive/warehouse/delta_demo.db/product_table_delta/_delta_log/.s3-optimization-2,.s3-optimization-2,0
dbfs:/user/hive/warehouse/delta_demo.db/product_table_delta/_delta_log/00000000000000000000.crc,00000000000000000000.crc,89
dbfs:/user/hive/warehouse/delta_demo.db/product_table_delta/_delta_log/00000000000000000000.json,00000000000000000000.json,3287
dbfs:/user/hive/warehouse/delta_demo.db/product_table_delta/_delta_log/00000000000000000001.crc,00000000000000000001.crc,89
dbfs:/user/hive/warehouse/delta_demo.db/product_table_delta/_delta_log/00000000000000000001.json,00000000000000000001.json,3414
dbfs:/user/hive/warehouse/delta_demo.db/product_table_delta/_delta_log/00000000000000000002.crc,00000000000000000002.crc,91
dbfs:/user/hive/warehouse/delta_demo.db/product_table_delta/_delta_log/00000000000000000002.json,00000000000000000002.json,2700
dbfs:/user/hive/warehouse/delta_demo.db/product_table_delta/_delta_log/00000000000000000003.crc,00000000000000000003.crc,91


In [47]:
%fs

head dbfs:/user/hive/warehouse/delta_demo.db/product_table_delta/_delta_log/00000000000000000006.json

{"commitInfo":{"timestamp":1593340899589,"userId":"1583409609571278","userName":"techieeisland@gmail.com","operation":"WRITE","operationParameters":{"mode":"Append","partitionBy":"[]"},"notebook":{"notebookId":"891795827403111"},"clusterId":"0628-102648-pick544","readVersion":5,"isolationLevel":"WriteSerializable","isBlindAppend":true,"operationMetrics":{"numFiles":"5","numOutputBytes":"6618","numOutputRows":"4","numParts":"0"}}}
{"metaData":{"id":"59535559-84e1-46da-a566-06c753305e5a","format":{"provider":"parquet","options":{}},"schemaString":"{\"type\":\"struct\",\"fields\":[{\"name\":\"product_id\",\"type\":\"long\",\"nullable\":true,\"metadata\":{}},{\"name\":\"product_name\",\"type\":\"string\",\"nullable\":true,\"metadata\":{}},{\"name\":\"row_flag\",\"type\":\"string\",\"nullable\":true,\"metadata\":{}},{\"name\":\"stocks_left\",\"type\":\"long\",\"nullable\":true,\"metadata\":{}},{\"name\":\"new_col\",\"type\":\"string\",\"nullable\":true,\"metadata\":{}}]}","partitionColumns":[],"configuration":{},"createdTime":1593330009451}}
{"add":{"path":"part-00000-1baa5b2e-f525-45d6-9ea4-ae79ba1bdceb-c000.snappy.parquet","partitionValues":{},"size":645,"modificationTime":1593340900000,"dataChange":true,"stats":"{\"numRecords\":0,\"minValues\":{},\"maxValues\":{},\"nullCount\":{}}"}}
{"add":{"path":"part-00001-d26756df-406f-4db0-a66f-ab5ccf574d96-c000.snappy.parquet","partitionValues":{},"size":1498,"modificationTime":1593340900000,"dataChange":true,"stats":"{\"numRecords\":1,\"minValues\":{\"new_col\":\"new_1\",\"product_id\":1001,\"product_name\":\"Apple Mac Book\",\"row_flag\":\"I\",\"stocks_left\":11},\"maxValues\":{\"new_col\":\"new_1\",\"product_id\":1001,\"product_name\":\"Apple Mac Book\",\"row_flag\":\"I\",\"stocks_left\":11},\"nullCount\":{\"new_col\":0,\"product_id\":0,\"product_name\":0,\"row_flag\":0,\"stocks_left\":0}}"}}
{"add":{"path":"part-00003-a9cd7ebe-f8b6-4f0c-9344-997cab24eb3b-c000.snappy.parquet","partitionValues":{},"size":1507,"modificationTime":1593340900000,"dataChange":true,"stats":"{\"numRecords\":1,\"minValues\":{\"new_col\":\"new_2\",\"product_id\":1002,\"product_name\":\"Apple iPhone 11\",\"row_flag\":\"I\",\"stocks_left\":22},\"maxValues\":{\"new_col\":\"new_2\",\"product_id\":1002,\"product_name\":\"Apple iPhone 11\",\"row_flag\":\"I\",\"stocks_left\":22},\"nullCount\":{\"new_col\":0,\"product_id\":0,\"product_name\":0,\"row_flag\":0,\"stocks_left\":0}}"}}
{"add":{"path":"part-00005-db28f365-bae8-43c1-89ba-85bf77fc7932-c000.snappy.parquet","partitionValues":{},"size":1480,"modificationTime":1593340900000,"dataChange":true,"stats":"{\"numRecords\":1,\"minValues\":{\"new_col\":\"new_3\",\"product_id\":1003,\"product_name\":\"Redmi Note 4\",\"row_flag\":\"I\",\"stocks_left\":33},\"maxValues\":{\"new_col\":\"new_3\",\"product_id\":1003,\"product_name\":\"Redmi Note 4\",\"row_flag\":\"I\",\"stocks_left\":33},\"nullCount\":{\"new_col\":0,\"product_id\":0,\"product_name\":0,\"row_flag\":0,\"stocks_left\":0}}"}}
{"add":{"path":"part-00007-7ad2390f-b48f-422c-96e6-728bbc893028-c000.snappy.parquet","partitionValues":{},"size":1489,"modificationTime":1593340900000,"dataChange":true,"stats":"{\"numRecords\":1,\"minValues\":{\"new_col\":\"new_4\",\"product_id\":1004,\"product_name\":\"Dell Inspiron\",\"row_flag\":\"I\",\"stocks_left\":44},\"maxValues\":{\"new_col\":\"new_4\",\"product_id\":1004,\"product_name\":\"Dell Inspiron\",\"row_flag\":\"I\",\"stocks_left\":44},\"nullCount\":{\"new_col\":0,\"product_id\":0,\"product_name\":0,\"row_flag\":0,\"stocks_left\":0}}"}}

In [48]:
display(spark.read.json("dbfs:/user/hive/warehouse/delta_demo.db/product_table_delta/_delta_log/00000000000000000006.json"))

add,commitInfo,metaData
null,"List(0628-102648-pick544, true, WriteSerializable, List(891795827403111), WRITE, List(5, 6618, 4, 0), List(Append, []), 5, 1593340899589, 1583409609571278, techieeisland@gmail.com)",null
null,null,"List(1593330009451, List(parquet), 59535559-84e1-46da-a566-06c753305e5a, List(), {""type"":""struct"",""fields"":[{""name"":""product_id"",""type"":""long"",""nullable"":true,""metadata"":{}},{""name"":""product_name"",""type"":""string"",""nullable"":true,""metadata"":{}},{""name"":""row_flag"",""type"":""string"",""nullable"":true,""metadata"":{}},{""name"":""stocks_left"",""type"":""long"",""nullable"":true,""metadata"":{}},{""name"":""new_col"",""type"":""string"",""nullable"":true,""metadata"":{}}]})"
"List(true, 1593340900000, part-00000-1baa5b2e-f525-45d6-9ea4-ae79ba1bdceb-c000.snappy.parquet, 645, {""numRecords"":0,""minValues"":{},""maxValues"":{},""nullCount"":{}})",null,null
"List(true, 1593340900000, part-00001-d26756df-406f-4db0-a66f-ab5ccf574d96-c000.snappy.parquet, 1498, {""numRecords"":1,""minValues"":{""new_col"":""new_1"",""product_id"":1001,""product_name"":""Apple Mac Book"",""row_flag"":""I"",""stocks_left"":11},""maxValues"":{""new_col"":""new_1"",""product_id"":1001,""product_name"":""Apple Mac Book"",""row_flag"":""I"",""stocks_left"":11},""nullCount"":{""new_col"":0,""product_id"":0,""product_name"":0,""row_flag"":0,""stocks_left"":0}})",null,null
"List(true, 1593340900000, part-00003-a9cd7ebe-f8b6-4f0c-9344-997cab24eb3b-c000.snappy.parquet, 1507, {""numRecords"":1,""minValues"":{""new_col"":""new_2"",""product_id"":1002,""product_name"":""Apple iPhone 11"",""row_flag"":""I"",""stocks_left"":22},""maxValues"":{""new_col"":""new_2"",""product_id"":1002,""product_name"":""Apple iPhone 11"",""row_flag"":""I"",""stocks_left"":22},""nullCount"":{""new_col"":0,""product_id"":0,""product_name"":0,""row_flag"":0,""stocks_left"":0}})",null,null
"List(true, 1593340900000, part-00005-db28f365-bae8-43c1-89ba-85bf77fc7932-c000.snappy.parquet, 1480, {""numRecords"":1,""minValues"":{""new_col"":""new_3"",""product_id"":1003,""product_name"":""Redmi Note 4"",""row_flag"":""I"",""stocks_left"":33},""maxValues"":{""new_col"":""new_3"",""product_id"":1003,""product_name"":""Redmi Note 4"",""row_flag"":""I"",""stocks_left"":33},""nullCount"":{""new_col"":0,""product_id"":0,""product_name"":0,""row_flag"":0,""stocks_left"":0}})",null,null
"List(true, 1593340900000, part-00007-7ad2390f-b48f-422c-96e6-728bbc893028-c000.snappy.parquet, 1489, {""numRecords"":1,""minValues"":{""new_col"":""new_4"",""product_id"":1004,""product_name"":""Dell Inspiron"",""row_flag"":""I"",""stocks_left"":44},""maxValues"":{""new_col"":""new_4"",""product_id"":1004,""product_name"":""Dell Inspiron"",""row_flag"":""I"",""stocks_left"":44},""nullCount"":{""new_col"":0,""product_id"":0,""product_name"":0,""row_flag"":0,""stocks_left"":0}})",null,null


In [49]:
display(spark.read.json("dbfs:/user/hive/warehouse/delta_demo.db/product_table_delta/_delta_log/00000000000000000007.json"))

add,commitInfo,remove
null,"List(0628-102648-pick544, false, WriteSerializable, List(891795827403111), MERGE, List(4, 5583, 6, 0, 4, 4, 10, 10, 8, 0, 2, 0, 6), List((source_table.`row_flag` = 'D'), (source_table.`row_flag` = 'I'), (source_table.`product_id` = target_table.`product_id`), (source_table.`row_flag` = 'U')), 6, 1593341532506, 1583409609571278, techieeisland@gmail.com)",null
null,null,"List(true, 1593341532454, part-00005-db28f365-bae8-43c1-89ba-85bf77fc7932-c000.snappy.parquet)"
null,null,"List(true, 1593341532502, part-00007-7ad2390f-b48f-422c-96e6-728bbc893028-c000.snappy.parquet)"
null,null,"List(true, 1593341532502, part-00001-d26756df-406f-4db0-a66f-ab5ccf574d96-c000.snappy.parquet)"
null,null,"List(true, 1593341532502, part-00003-f5307a9d-8be6-42d9-a9af-0b276c81e1d0-c000.snappy.parquet)"
null,null,"List(true, 1593341532502, part-00007-88bcdf6d-cb6b-4013-adaf-d63acc344dc8-c000.snappy.parquet)"
null,null,"List(true, 1593341532502, part-00003-a9cd7ebe-f8b6-4f0c-9344-997cab24eb3b-c000.snappy.parquet)"
null,null,"List(true, 1593341532502, part-00005-8b1e9299-6d7d-4e81-8650-d7b3b6d72626-c000.snappy.parquet)"
null,null,"List(true, 1593341532502, part-00001-60eb97be-368d-4dca-9643-191e93e04c93-c000.snappy.parquet)"
"List(true, 1593341529000, part-00000-dabb379d-646b-416d-b4f7-e2840c0a096b-c000.snappy.parquet, 645, {""numRecords"":0,""minValues"":{},""maxValues"":{},""nullCount"":{}})",null,null


### Explain about scalable metadata handling